In [1]:
# import pandas and duckdb
import pandas as pd
import duckdb
from pathlib import Path

In [2]:
sql_query = """
SHOW TABLES;
"""

In [3]:
# create data folder if it doesn't exist
data_path = Path("data")
data_path.mkdir(parents=True, exist_ok=True)

with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(sql_query).df())

,name


In [4]:
# import data into duckdb
sql_query_import_parking_violation_codes = """
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto(
    'data/parking_violation_codes.csv',
    normalize_names=True
);
"""

sql_query_import_parking_violations_2023 = """
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
    'data/parking_violations_2023.csv',
    normalize_names=True
);
"""

In [6]:
# lets connect to the duckdb and run the import queries
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    conn.execute(sql_query_import_parking_violation_codes)
    conn.execute(sql_query_import_parking_violations_2023)
    display(conn.execute("SHOW TABLES;").df())

,name
0,parking_violation_codes
1,parking_violations_2023


In [13]:
# lets see what is in parking_violations_codes
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute("SELECT * FROM parking_violation_codes LIMIT 5;").df())

,code,definition,manhattan_96th_st_below,all_other_areas
0,1,FAILURE TO DISPLAY BUS PERMIT,515,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,515,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,515,515
3,4,BUS PARKING IN LOWER MANHATTAN,115,115
4,5,BUS LANE VIOLATION,250,250


In [8]:
# lets see what columns are in the parking_violations_2023 table
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute("SELECT * FROM parking_violations_2023 LIMIT 5;").df())

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,...,from_hours_in_effect,to_hours_in_effect,vehicle_color,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,no_standing_or_stopping_violation,hydrant_violation,double_parking_violation
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50,...,0700A,0400P,BLACK,<NA>,0,None,0,None,None,None
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,<NA>,...,None,None,GY,<NA>,2003,None,0,None,None,None
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,<NA>,...,None,None,WHI,<NA>,2022,None,0,None,None,None
3,9044582707,NY,PAS,2023-04-10,21,SUBN,SUBAR,T,20231217,79,...,0900A,1030A,GY,<NA>,2017,None,0,None,None,None
4,9041503330,NY,PAS,2023-03-21,21,4DSD,CHEVR,T,20250320,26,...,1100A,1230A,BK,<NA>,2018,None,0,None,None,None


The `DBT init` command is used to create a new dbt project. When you run this command, it auto-generates a new dbt project with all the necessary files and directories you need to get started. 

Run the following command in your terminal to create a new dbt project:

```bash
dbt init nyc_parking_violations
```

After running the command, it will show the following output, enter "1" to select duckdb as the database:

```bash
(venv) rellika@mackbook ~/D/C/DTB (main)> dbt init nyc_parking_violations                       (base) 
05:47:22  Running with dbt=1.11.2
05:47:22  Creating dbt configuration folder at /Users/rellika/.dbt
05:47:22  
Your new dbt project "nyc_parking_violations" was created!

For more information on how to configure the profiles.yml file,
please consult the dbt documentation here:

  https://docs.getdbt.com/docs/configure-your-profile

One more thing:

Need help? Don't hesitate to reach out to us via GitHub issues or on Slack:

  https://community.getdbt.com/

Happy modeling!

05:47:22  Setting up your profile.
Which database would you like to use?
[1] duckdb

(Don't see the one you want? https://docs.getdbt.com/docs/available-adapters)

Enter a number: 1
```

It will create a new directory called `nyc_parking_violations` with the following structure:

```
nyc_parking_violations/
analyses/
macros/
models/
seeds/
snapshots/
tests/
.gitignore
dbt_project.yml
README.md
```

- dbt_project.yml: TThe DBT project YAML file is a configuration file used in DBT projects. It stands for `Yet Another Markup Language` and is used to summarize configurations for your project. This file contains key sections that inform your DBT project where to look for various components and what actions to take. It includes details like the project name, version, profiles, paths for models, tests, seeds, macros, and how models are materialized. This file is essential for setting up and managing your DBT project.

We need also to set up the `profiles.yml` file to connect to duckdb. `cd` into `nyc_parking_violations`, and then create a file called `profiles.yml` inside that folder with the following content:

```yaml
default:
    outputs:
        dev:
            type: duckdb
    target: dev
```

**DBT COMMANDS TO KNOW**

- `dbt debug`: This command is used to test the connection between dbt and your data warehouse. It checks if dbt can successfully connect to the database specified in your profiles.yml file and verifies that the configuration is correct. Running this command helps ensure that your dbt setup is properly configured before you start running models or other dbt commands.
- `dbt compile`: This command compiles your dbt models into executable SQL files without actually running them against the database. It processes your model files, applies any Jinja templating, and generates the final SQL code that would be executed. This is useful for checking the generated SQL and ensuring that your models are correctly defined before executing them.
- `dbt run`: This command executes the compiled SQL models against your data warehouse. It runs the SQL code generated by dbt to create or update tables and views in your database based on the models defined in your dbt project. This is the primary command used to build and transform data within your data warehouse using dbt.

In [10]:
# select * from ref_model
sql_query_ref_model = """
SELECT * FROM ref_model;
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(sql_query_ref_model).df())

,violation_count
0,97


In [11]:
# lets see the tables again
sql_show_tables = """
SHOW TABLES;
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(sql_show_tables).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,parking_violation_codes
4,parking_violations_2023
5,ref_model


In [12]:
# lets view our bronze data from parking_violations_2023
sql_view_bronze_data = """
SELECT * FROM parking_violations_2023 LIMIT 5;
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(sql_view_bronze_data).df())

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,...,from_hours_in_effect,to_hours_in_effect,vehicle_color,unregistered_vehicle,vehicle_year,meter_number,feet_from_curb,no_standing_or_stopping_violation,hydrant_violation,double_parking_violation
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50,...,0700A,0400P,BLACK,<NA>,0,None,0,None,None,None
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,<NA>,...,None,None,GY,<NA>,2003,None,0,None,None,None
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,<NA>,...,None,None,WHI,<NA>,2022,None,0,None,None,None
3,9044582707,NY,PAS,2023-04-10,21,SUBN,SUBAR,T,20231217,79,...,0900A,1030A,GY,<NA>,2017,None,0,None,None,None
4,9041503330,NY,PAS,2023-03-21,21,4DSD,CHEVR,T,20250320,26,...,1100A,1230A,BK,<NA>,2018,None,0,None,None,None


In [ ]:
# lets see the tables again
manhattan_violation_codes = """
SELECT
        violation_code,                          -- Unique identifier for the violation type
        definition,                              -- Description of the violation
        TRUE AS is_manhattan_96th_st_below,      -- Flag indicating this row is for Manhattan pricing
        manhattan_96th_st_below AS fee_usd       -- Fine amount for Manhattan 96th St & below
    FROM
        'bronze_parking_violation_codes' 
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(manhattan_violation_codes).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,1,FAILURE TO DISPLAY BUS PERMIT,True,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,True,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,True,515
3,4,BUS PARKING IN LOWER MANHATTAN,True,115
4,5,BUS LANE VIOLATION,True,250
...,...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,True,65
93,96,RAILROAD CROSSING,True,95
94,97,VACANT LOT,True,65
95,98,OBSTRUCTING DRIVEWAY,True,95


In [16]:
# lets see the tables again
all_other_violation_codes = """
SELECT
        violation_code,                          -- Unique identifier for the violation type
        definition,                              -- Description of the violation
        FALSE AS is_manhattan_96th_st_below,     -- Flag indicating this row is NOT for Manhattan pricing
        all_other_areas AS fee_usd               -- Fine amount for all other areas
    FROM
        'bronze_parking_violation_codes' 
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(all_other_violation_codes).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,1,FAILURE TO DISPLAY BUS PERMIT,False,515
1,2,NO OPERATOR NAM/ADD/PH DISPLAY,False,515
2,3,UNAUTHORIZED PASSENGER PICK-UP,False,515
3,4,BUS PARKING IN LOWER MANHATTAN,False,115
4,5,BUS LANE VIOLATION,False,250
...,...,...,...,...
92,93,REMOVE/REPLACE FLAT TIRE,False,65
93,96,RAILROAD CROSSING,False,95
94,97,VACANT LOT,False,45
95,98,OBSTRUCTING DRIVEWAY,False,95


In [22]:
# CTE example - combining Manhattan and all other areas violation codes
combined_violation_codes = """
WITH manhattan_violation_codes AS (
    SELECT
        code AS violation_code,
        definition,
        TRUE AS is_manhattan_96th_st_below,
        manhattan_96th_st_below AS fee_usd
    FROM parking_violation_codes
),
all_other_violation_codes AS (
    SELECT
        code AS violation_code,
        definition,
        FALSE AS is_manhattan_96th_st_below,
        all_other_areas AS fee_usd
    FROM parking_violation_codes
)

SELECT * FROM manhattan_violation_codes
UNION ALL
SELECT * FROM all_other_violation_codes
ORDER BY violation_code ASC
"""

with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(combined_violation_codes).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,1,FAILURE TO DISPLAY BUS PERMIT,False,515
1,1,FAILURE TO DISPLAY BUS PERMIT,True,515
2,2,NO OPERATOR NAM/ADD/PH DISPLAY,False,515
3,2,NO OPERATOR NAM/ADD/PH DISPLAY,True,515
4,3,UNAUTHORIZED PASSENGER PICK-UP,False,515
...,...,...,...,...
189,97,VACANT LOT,True,65
190,98,OBSTRUCTING DRIVEWAY,False,95
191,98,OBSTRUCTING DRIVEWAY,True,95
192,99,OTHER,False,105


In [18]:
silver_parking_violation = """
    SELECT
    summons_number,
    registration_state,
    plate_type,
    issue_date,
    violation_code,
    vehicle_body_type,
    vehicle_make,
    issuing_agency,
    vehicle_expiration_date,
    violation_location,
    violation_precinct,
    issuer_precinct,
    issuer_code,
    issuer_command,
    issuer_squad,
    violation_time,
    violation_county,
    violation_legal_code,
    vehicle_color,
    vehicle_year,
    CASE WHEN
        violation_county == 'MN'
        THEN TRUE
        ELSE FALSE
        END AS is_manhattan_96th_st_below
FROM
    bronze_parking_violations
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(silver_parking_violation).df())

,summons_number,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,vehicle_expiration_date,violation_location,...,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,vehicle_color,vehicle_year,is_manhattan_96th_st_below
0,9010912681,CA,PAS,2022-10-11,17,SUBN,FORD,T,20220788,50,...,50,365910,T201,D,0825A,BX,<NA>,BLACK,0,False
1,4858762841,NY,PAS,2023-08-21,36,4DSD,HONDA,V,0,<NA>,...,0,0,None,None,1036A,BK,True,GY,2003,False
2,4854645684,FL,PAS,2023-07-26,36,UT,BMW,V,0,<NA>,...,0,0,None,None,0126P,QN,True,WHI,2022,False
3,9044582707,NY,PAS,2023-04-10,21,SUBN,SUBAR,T,20231217,79,...,79,370326,T301,H,0924A,K,<NA>,GY,2017,False
4,9041503330,NY,PAS,2023-03-21,21,4DSD,CHEVR,T,20250320,26,...,26,373097,T103,C,1155A,NY,<NA>,BK,2018,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,9069356715,NY,COM,2023-08-07,21,VAN,FORD,T,20240831,30,...,30,361887,T103,II,1121A,NY,<NA>,YW,2016,False
99996,4802425399,NY,OMS,2022-09-25,36,SUBN,CHEVR,V,0,<NA>,...,0,0,None,None,0214P,BX,True,WH,2021,False
99997,4809109689,NY,ORG,2022-10-28,36,SUBN,NISSA,V,0,<NA>,...,0,0,None,None,0915A,MN,True,GY,2020,True
99998,4834558370,NY,PAS,2023-04-04,36,4DSD,LEXUS,V,0,<NA>,...,0,0,None,None,0416P,BK,True,BK,2002,False


In [23]:
# lets see all the tables again

# lets see the tables again
show_tables = """
SHOW TABLES;
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(show_tables).df())

,name
0,bronze_parking_violation_codes
1,bronze_parking_violations
2,first_model
3,parking_violation_codes
4,parking_violations_2023
5,ref_model
6,silver_parking_violation_codes
7,silver_parking_violations
8,silver_violation_tickets
9,silver_violation_vehicles


In [24]:
# lets see what is in silver_parking_violation_codes
data_silver_parking_violation_codes = """
SELECT * FROM silver_parking_violation_codes LIMIT 5;
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(data_silver_parking_violation_codes).df())

,violation_code,definition,is_manhattan_96th_st_below,fee_usd
0,1,FAILURE TO DISPLAY BUS PERMIT,False,515
1,1,FAILURE TO DISPLAY BUS PERMIT,True,515
2,2,NO OPERATOR NAM/ADD/PH DISPLAY,False,515
3,2,NO OPERATOR NAM/ADD/PH DISPLAY,True,515
4,3,UNAUTHORIZED PASSENGER PICK-UP,True,515


In [25]:
# lets see what is in silver_violation_tickets
data_silver_violation_tickets = """
SELECT * FROM silver_violation_tickets LIMIT 5;
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(data_silver_violation_tickets).df())

,summons_number,issue_date,violation_code,is_manhattan_96th_st_below,issuing_agency,violation_location,violation_precinct,issuer_precinct,issuer_code,issuer_command,issuer_squad,violation_time,violation_county,violation_legal_code,fee_usd
0,9010912681,2022-10-11,17,False,T,50,50,50,365910,T201,D,0825A,BX,<NA>,95
1,4858762841,2023-08-21,36,False,V,<NA>,0,0,0,None,None,1036A,BK,True,50
2,4854645684,2023-07-26,36,False,V,<NA>,0,0,0,None,None,0126P,QN,True,50
3,9044582707,2023-04-10,21,False,T,79,79,79,370326,T301,H,0924A,K,<NA>,65
4,9041503330,2023-03-21,21,False,T,26,26,26,373097,T103,C,1155A,NY,<NA>,65


**Materializations in DBT**
In dbt, materializations refer to the different ways that dbt can create and manage database objects (like tables and views) based on your models. Materializations determine how the data is stored and accessed in your data warehouse. Here are some common types of materializations in dbt:
- `Table`: This materialization creates a physical table in the database. When you run dbt with a model set to "table" materialization, it will create a new table or replace an existing one with the results of the model's SQL query.
- `View`: This materialization creates a database view instead of a physical table. A view is a virtual table that is defined by a SQL query. When you run dbt with a model set to "view" materialization, it will create or replace a view in the database based on the model's SQL query.
- `incremental`: This materialization allows you to build tables incrementally. Instead of recreating the entire table each time, dbt will only add new or updated data based on a defined unique key. This is useful for large datasets where you want to optimize performance and reduce processing time.
- `ephemeral`: This materialization does not create any physical database objects. Instead, it creates temporary tables
- `materialized views`: Some databases support materialized views, which are physical copies of query results that can be refreshed periodically. dbt can create materialized views if the underlying database supports them.

In [26]:
# lets see the failed tests
failed_tests = """
  SELECT * FROM "nyc_parking_violations"."main_dbt_test__audit"."violation_codes_revenue"
"""
with duckdb.connect("data/nyc_parking_violations.db") as conn:
    display(conn.execute(failed_tests).df())

,violation_code,total_revenue_usd
0,41,0.0


In [27]:
# create a production database
# import data into duckdb
PRODUCTION_import_parking_violation_codes = """
CREATE OR REPLACE TABLE parking_violation_codes AS
SELECT *
FROM read_csv_auto(
    'data/parking_violation_codes.csv',
    normalize_names=True
);
"""

PRODUCTION_import_parking_violations_2023 = """
CREATE OR REPLACE TABLE parking_violations_2023 AS
SELECT *
FROM read_csv_auto(
    'data/parking_violations_2023.csv',
    normalize_names=True
);
"""

In [29]:
# run the imports
# lets connect to the duckdb and run the import queries
with duckdb.connect("data/prod_nyc_parking_violations.db") as conn:
    conn.execute(PRODUCTION_import_parking_violation_codes)
    conn.execute(PRODUCTION_import_parking_violations_2023)
    display(conn.execute("SHOW TABLES;").df()) 

,name
0,parking_violation_codes
1,parking_violations_2023
